In [1]:
import os
from detectors import *
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

D:\Anaconda\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
path = "C:/Users/luiz_/Projects/DataStreams/Quantification-in-Data-Streams/datasets"
files = os.listdir(path)
datasets = [x[:-4] for x in files]

In [3]:
clf = RandomForestClassifier()
parameters = {'n_estimators':[100, 200, 300, 400, 500]}
model = GridSearchCV(estimator = clf,
                    param_grid = parameters,
                    scoring = 'accuracy',
                    cv = 10,
                    n_jobs = -1)

In [4]:
table = pd.DataFrame(index=[datasets])
window_size = 300 #window parameter to build the images for comparison
train_size = 499
table

""
2CDT
2CHT
FG_2C_2D
GEARS_2C_2D
MG_2C_2D
UG_2C_2D
UG_2C_3D
UG_2C_5D


In [ ]:
for file in files:
    stream = pd.read_csv(path + "/" + file, header=0, index_col=False,sep=',')
    stream.iloc[:, -1].replace(1, int(0), inplace=True)
    stream.iloc[:, -1].replace(2, int(1), inplace=True)
    train = stream.iloc[:train_size].reset_index(drop=True)
    test = stream.iloc[train_size:].reset_index(drop=True)

    print(f"Dataset: {file[:-4]}")
    for detector in ["IKS", "IBDD", "WRS", "ADWIN", "BASELINE","TOPLINE"]:
        if detector == "IKS":
            threshold = 1.95
            [drifts, mean_acc, time] = IKS(train, test, window_size, threshold, model)
        elif detector == "IBDD":
            epsilon = 3
            [drifts, acc_vector, mean_acc, time] = IBDD(train, test, window_size, epsilon, model)
        elif detector == "WRS":
            threshold = 0.001
            [drifts, acc_vector, mean_acc, time] = WRS(train, test, window_size, threshold, model)
        elif detector == "ADWIN":
            [drifts, acc_vector, mean_acc, time] = Adwin(train, test, window_size, model)
        elif detector == "TOPLINE":
            [mean_acc, acc_vector, time] = topline_classifier(train, test, window_size, model)
        elif detector == "BASELINE":
            [mean_acc, acc_vector, time] = baseline_classifier(train, test, model)
        print("=======================")
            
        table.loc[[file[:-4]], [detector]] = round(mean_acc)
table

    

Dataset: 2CDT
IKS Running...
1xample 1/15500 - drifts: []
0    159
1    141
Name: count, dtype: int64
   IKS  IKS-CC  IKS-ACC  IKS-MS  IKS-SORD
0  1.0     1.0     0.97    0.98      0.91
1xample 2/15500 - drifts: []
0    159
1    141
Name: count, dtype: int64
   IKS  IKS-CC  IKS-ACC  IKS-MS  IKS-SORD
0  1.0     1.0     0.97    0.98      0.91
1  1.0     1.0     0.99    0.97      0.90
1xample 3/15500 - drifts: []
0    159
1    141
Name: count, dtype: int64
   IKS  IKS-CC  IKS-ACC  IKS-MS  IKS-SORD
0  1.0     1.0     0.97    0.98      0.91
1  1.0     1.0     0.99    0.97      0.90
2  1.0     1.0     0.99    1.00      0.87
1xample 4/15500 - drifts: []
0    159
1    141
Name: count, dtype: int64
   IKS  IKS-CC  IKS-ACC  IKS-MS  IKS-SORD
0  1.0     1.0     0.97    0.98      0.91
1  1.0     1.0     0.99    0.97      0.90
2  1.0     1.0     0.99    1.00      0.87
3  1.0     1.0     0.98    0.98      0.90
1xample 5/15500 - drifts: []
0    158
1    142
Name: count, dtype: int64
   IKS  IKS-CC  IK

# INCREMENTAL KOLMOGOROV-SMIRNOV (IKS):

In [ ]:
FILENAME_TRAIN = 'benchmark_real/Yoga_TRAIN.data'
FILENAME_TEST = 'benchmark_real/Yoga_TEST.data'
threshold = 1.12
[drifts, acc_vector, mean_acc, time] = IKS(FILENAME_TRAIN, FILENAME_TEST, window_size, threshold)

# IMAGE-BASED DRIFT DETECTION (IBDD):

In [ ]:
epsilon = 3 #number of MSD values above/below threshold
[drifts, acc_vector, mean_acc, time] = IBDD(FILENAME_TRAIN, FILENAME_TEST, window_size, epsilon)

# BASELINE:

In [ ]:
[mean_acc, acc_vector, time] = baseline_classifier(FILENAME_TRAIN, FILENAME_TEST)

# TOPLINE:

In [ ]:
[mean_acc, acc_vector, time] = topline_classifier(FILENAME_TRAIN, FILENAME_TEST, window_size)

# WILCOXON-RANK SUM (WRS):

In [ ]:
threshold = 0.001
[drifts, acc_vector, mean_acc, time] = wrs_test(FILENAME_TRAIN, FILENAME_TEST, window_size, threshold)